In [1]:
library(tidyverse)
library(Seurat)
library(targets)

source('../01_milo_cellbender/milo_cellbender.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [2]:
other_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_other_01') %>% 
    set_labels_to_lvl1 %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
other_clusters

[1] "Tanycytes"         "Oligodendrocytes"  "OPC"              
[4] "Astrocytes"        "Ependymal_cells"   "Microglia"        
[7] "Pericytes"         "Endothelial_cells" "VLMC"

In [3]:
neuron_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_neuron_00') %>% 
    set_labels_to_lvl1 %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
neuron_clusters 

[1] "Sst__Pthlh"          "Nfix__Nfib"          "Agrp"               
 [4] "Sim1"                "Fez1"                "Pou6f2__Prkcq__Whrn"
 [7] "Dlk1"                "Slc17a6__Trhr"       "Esr1"               
[10] "Tmem215__Dach2"      "Sst__Nts"            "Sst__Unc13c"        
[13] "Irx5__Lmx1a"         "Htr3b"               "Hdc"                
[16] "Lepr89"              "Nfib_1"              "Ppp1r17__Tafa4"     
[19] "Lepr67"              "Arx__Nr5a2"          "Kiss1__Tac2"        
[22] "Met__Prlh"           "Met__Pappa2"         "Trh__Lef1"          
[25] "Gpr50__Tac2"         "Tmem215__Greb1"      "Ghrh"               
[28] "unassigned__2__"     "Lepr15"              "Pomc__Ttr"          
[31] "Tbx19__Pirt"         "Pomc__Glipr1"        "Pomc__Anxa2"        
[34] "Npsr1__Epha3"        "Trh__Cxcl12"         "Nfib_2"             
[37] "Nfix__Nfib__Lhx6"    "Reln__Lef1"          "Th__Nfib"           
[40] "Ppp1r17"             "Th__Slc6a3"          "Nfib_3"             
[43] "Ppp1r17__Ltbp1"      "Lpar1_oligo"         "Nfix__Nfib__Hgf"

In [112]:
day_splits = tibble(object = rlang::syms(c("exp_labelled_other", "exp_labelled_neuron")))
# days = tibble(day = c("Day5", "Day14", "all"))
days = tibble(day = c("Day5", "Day14"))
day_splits = tidyr::crossing(day_splits, days)
day_splits

object,day
<list>,<chr>
exp_labelled_other,Day14
exp_labelled_other,Day5
exp_labelled_neuron,Day14
exp_labelled_neuron,Day5


In [113]:
other_clusters_tibble = tibble(label = other_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'other'))


neuron_clusters_tibble = tibble(label = neuron_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'neuron'))

clusters_tibble = rbind(other_clusters_tibble, neuron_clusters_tibble)
clusters_tibble = clusters_tibble %>% 
mutate(cluster = label) %>%
rowwise %>%
mutate(label = case_when(day != "all" ~ paste0(c(day, as.character(label)), collapse='.'),
                          TRUE ~ paste0(label, collapse='.'))) %>%
filter(!str_detect(cluster, 'unassigned'))
clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<chr>
Day14.Astrocytes,exp_labelled_other,Day14,Astrocytes
Day5.Astrocytes,exp_labelled_other,Day5,Astrocytes
Day14.Endothelial_cells,exp_labelled_other,Day14,Endothelial_cells
Day5.Endothelial_cells,exp_labelled_other,Day5,Endothelial_cells
Day14.Ependymal_cells,exp_labelled_other,Day14,Ependymal_cells
Day5.Ependymal_cells,exp_labelled_other,Day5,Ependymal_cells
Day14.Microglia,exp_labelled_other,Day14,Microglia
Day5.Microglia,exp_labelled_other,Day5,Microglia
Day14.Oligodendrocytes,exp_labelled_other,Day14,Oligodendrocytes


In [114]:
qs::qsave(clusters_tibble, 'clusters_tibble_lvl1.qs')

In [115]:
dims = c(seq(5, 30, 5), seq(30, 50, 10)) %>% unique
k.anchor = c(5, 25, 50)
k.weight = c(20, 50, 100, 150, 200)
nfeats = c(2000, 5000, 10000)
batch = c("batch", "orig.batch")

In [116]:
param_tibble = tidyr::crossing(dims = dims,
                                k.anchor = k.anchor,
                                k.weight = k.weight,
                                nfeats = nfeats,
                                batch = batch) %>%
rowwise %>%
mutate(hash = digest::digest(c(dims, k.anchor, k.weight, nfeats, batch), algo = 'md5') #%>% 
       #str_sub(-6, -1)
      ) %>%
ungroup 

param_tibble

dims,k.anchor,k.weight,nfeats,batch,hash
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
5,5,20,2000,batch,60eb74d29896f11015fba443f8205d27
5,5,20,2000,orig.batch,035beb6b7204d97fa39c3499fc17fe4f
5,5,20,5000,batch,33eb0ae188b2e87cb6427b0904f3652d
5,5,20,5000,orig.batch,79bb36c61d501acdc4bd3e65deea7d76
5,5,20,10000,batch,3643fc71e93c38bd47ba813afb74f25d
5,5,20,10000,orig.batch,f54852f0417d01a1cd128416f58916b1
5,5,50,2000,batch,3eefd5739ca9bb8a316b7466d14488bd
5,5,50,2000,orig.batch,3248f064a1540045d7c511efb647c903
5,5,50,5000,batch,2a87df7ac54c35ad07879d50cef5a5db


In [117]:
clusters_tibble = tidyr::crossing(clusters_tibble, param_tibble) %>%
mutate(old.label = label) %>%
rowwise %>%
mutate(label = paste0(label, '_', hash))

clusters_tibble

label,object,day,cluster,dims,k.anchor,k.weight,nfeats,batch,hash,old.label
<chr>,<list>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Day14.Agrp_60eb74d29896f11015fba443f8205d27,exp_labelled_neuron,Day14,Agrp,5,5,20,2000,batch,60eb74d29896f11015fba443f8205d27,Day14.Agrp
Day14.Agrp_035beb6b7204d97fa39c3499fc17fe4f,exp_labelled_neuron,Day14,Agrp,5,5,20,2000,orig.batch,035beb6b7204d97fa39c3499fc17fe4f,Day14.Agrp
Day14.Agrp_33eb0ae188b2e87cb6427b0904f3652d,exp_labelled_neuron,Day14,Agrp,5,5,20,5000,batch,33eb0ae188b2e87cb6427b0904f3652d,Day14.Agrp
Day14.Agrp_79bb36c61d501acdc4bd3e65deea7d76,exp_labelled_neuron,Day14,Agrp,5,5,20,5000,orig.batch,79bb36c61d501acdc4bd3e65deea7d76,Day14.Agrp
Day14.Agrp_3643fc71e93c38bd47ba813afb74f25d,exp_labelled_neuron,Day14,Agrp,5,5,20,10000,batch,3643fc71e93c38bd47ba813afb74f25d,Day14.Agrp
Day14.Agrp_f54852f0417d01a1cd128416f58916b1,exp_labelled_neuron,Day14,Agrp,5,5,20,10000,orig.batch,f54852f0417d01a1cd128416f58916b1,Day14.Agrp
Day14.Agrp_3eefd5739ca9bb8a316b7466d14488bd,exp_labelled_neuron,Day14,Agrp,5,5,50,2000,batch,3eefd5739ca9bb8a316b7466d14488bd,Day14.Agrp
Day14.Agrp_3248f064a1540045d7c511efb647c903,exp_labelled_neuron,Day14,Agrp,5,5,50,2000,orig.batch,3248f064a1540045d7c511efb647c903,Day14.Agrp
Day14.Agrp_2a87df7ac54c35ad07879d50cef5a5db,exp_labelled_neuron,Day14,Agrp,5,5,50,5000,batch,2a87df7ac54c35ad07879d50cef5a5db,Day14.Agrp


In [118]:
qs::qsave(clusters_tibble, 'clusters_tibble_lvl1.qs')

In [119]:
erroring_targs = run_until_success(function () qs::qread('erroring_targs.qs'),
                  function () c("blah"))

In [120]:
erroring_targs = tar_meta() %>%
filter(!is.na(error)) %>%
pull(name) %>%
str_replace('^obj_', '') %>%
c(erroring_targs, .)

qs::qsave(erroring_targs, "erroring_targs.qs")

In [121]:
clusters_tibble = clusters_tibble %>%
filter(!(label %in% erroring_targs))

clusters_tibble

label,object,day,cluster,dims,k.anchor,k.weight,nfeats,batch,hash,old.label
<chr>,<list>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Day14.Agrp_60eb74d29896f11015fba443f8205d27,exp_labelled_neuron,Day14,Agrp,5,5,20,2000,batch,60eb74d29896f11015fba443f8205d27,Day14.Agrp
Day14.Agrp_035beb6b7204d97fa39c3499fc17fe4f,exp_labelled_neuron,Day14,Agrp,5,5,20,2000,orig.batch,035beb6b7204d97fa39c3499fc17fe4f,Day14.Agrp
Day14.Agrp_33eb0ae188b2e87cb6427b0904f3652d,exp_labelled_neuron,Day14,Agrp,5,5,20,5000,batch,33eb0ae188b2e87cb6427b0904f3652d,Day14.Agrp
Day14.Agrp_79bb36c61d501acdc4bd3e65deea7d76,exp_labelled_neuron,Day14,Agrp,5,5,20,5000,orig.batch,79bb36c61d501acdc4bd3e65deea7d76,Day14.Agrp
Day14.Agrp_3643fc71e93c38bd47ba813afb74f25d,exp_labelled_neuron,Day14,Agrp,5,5,20,10000,batch,3643fc71e93c38bd47ba813afb74f25d,Day14.Agrp
Day14.Agrp_f54852f0417d01a1cd128416f58916b1,exp_labelled_neuron,Day14,Agrp,5,5,20,10000,orig.batch,f54852f0417d01a1cd128416f58916b1,Day14.Agrp
Day14.Agrp_3eefd5739ca9bb8a316b7466d14488bd,exp_labelled_neuron,Day14,Agrp,5,5,50,2000,batch,3eefd5739ca9bb8a316b7466d14488bd,Day14.Agrp
Day14.Agrp_3248f064a1540045d7c511efb647c903,exp_labelled_neuron,Day14,Agrp,5,5,50,2000,orig.batch,3248f064a1540045d7c511efb647c903,Day14.Agrp
Day14.Agrp_2a87df7ac54c35ad07879d50cef5a5db,exp_labelled_neuron,Day14,Agrp,5,5,50,5000,batch,2a87df7ac54c35ad07879d50cef5a5db,Day14.Agrp


In [122]:
priority_clusters = c(".*Sst.*",
                      ".*Pomc.*",
                      "Agrp",
                      "Htr3b",
                      "Tanycytes",
                      "Oligodendrocytes",
                      "OPC",
                      "Astrocytes",
                      "Ependymal_cells",
                      "Microglia")

priority_clusters = c("Astrocytes")

priority_clusters_str = paste0(priority_clusters, collapse = '|')
priority_clusters_str


clusters_tibble = clusters_tibble %>%
filter(str_detect(cluster, priority_clusters_str)) 

clusters_tibble

[1] "Astrocytes"

label,object,day,cluster,dims,k.anchor,k.weight,nfeats,batch,hash,old.label
<chr>,<list>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Day14.Astrocytes_60eb74d29896f11015fba443f8205d27,exp_labelled_other,Day14,Astrocytes,5,5,20,2000,batch,60eb74d29896f11015fba443f8205d27,Day14.Astrocytes
Day14.Astrocytes_035beb6b7204d97fa39c3499fc17fe4f,exp_labelled_other,Day14,Astrocytes,5,5,20,2000,orig.batch,035beb6b7204d97fa39c3499fc17fe4f,Day14.Astrocytes
Day14.Astrocytes_33eb0ae188b2e87cb6427b0904f3652d,exp_labelled_other,Day14,Astrocytes,5,5,20,5000,batch,33eb0ae188b2e87cb6427b0904f3652d,Day14.Astrocytes
Day14.Astrocytes_79bb36c61d501acdc4bd3e65deea7d76,exp_labelled_other,Day14,Astrocytes,5,5,20,5000,orig.batch,79bb36c61d501acdc4bd3e65deea7d76,Day14.Astrocytes
Day14.Astrocytes_3643fc71e93c38bd47ba813afb74f25d,exp_labelled_other,Day14,Astrocytes,5,5,20,10000,batch,3643fc71e93c38bd47ba813afb74f25d,Day14.Astrocytes
Day14.Astrocytes_f54852f0417d01a1cd128416f58916b1,exp_labelled_other,Day14,Astrocytes,5,5,20,10000,orig.batch,f54852f0417d01a1cd128416f58916b1,Day14.Astrocytes
Day14.Astrocytes_3eefd5739ca9bb8a316b7466d14488bd,exp_labelled_other,Day14,Astrocytes,5,5,50,2000,batch,3eefd5739ca9bb8a316b7466d14488bd,Day14.Astrocytes
Day14.Astrocytes_3248f064a1540045d7c511efb647c903,exp_labelled_other,Day14,Astrocytes,5,5,50,2000,orig.batch,3248f064a1540045d7c511efb647c903,Day14.Astrocytes
Day14.Astrocytes_2a87df7ac54c35ad07879d50cef5a5db,exp_labelled_other,Day14,Astrocytes,5,5,50,5000,batch,2a87df7ac54c35ad07879d50cef5a5db,Day14.Astrocytes


In [123]:
clusters_tibble %>% distinct %>% dim
clusters_tibble %>% distinct(old.label, hash) %>% dim

[1] 1423   11

[1] 1423    2

In [124]:
clusters_tibble %>% filter(hash == '9f00')

label,object,day,cluster,dims,k.anchor,k.weight,nfeats,batch,hash,old.label
<chr>,<list>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>


In [125]:
qs::qsave(clusters_tibble, 'clusters_tibble_lvl1.qs')